In [1]:
import pandas as pd
import numpy as np

# Load raw tables
accounts = pd.read_csv("../data/raw/ravenstack_accounts.csv")
subs = pd.read_csv("../data/raw/ravenstack_subscriptions.csv")
usage = pd.read_csv("../data/raw/ravenstack_feature_usage.csv")
tickets = pd.read_csv("../data/raw/ravenstack_support_tickets.csv")
churn = pd.read_csv("../data/raw/ravenstack_churn_events.csv")

# Parse dates
accounts["signup_date"] = pd.to_datetime(accounts["signup_date"])

subs["start_date"] = pd.to_datetime(subs["start_date"])
subs["end_date"] = pd.to_datetime(subs["end_date"])

usage["usage_date"] = pd.to_datetime(usage["usage_date"])

tickets["submitted_at"] = pd.to_datetime(tickets["submitted_at"])
tickets["closed_at"] = pd.to_datetime(tickets["closed_at"])

churn["churn_date"] = pd.to_datetime(churn["churn_date"])

# Quick check
accounts.head()

,account_id,account_name,industry,country,signup_date,referral_source,plan_tier,seats,is_trial,churn_flag
0,A-2e4581,Company_0,EdTech,US,2024-10-16,partner,Basic,9,False,False
1,A-43a9e3,Company_1,FinTech,IN,2023-08-17,other,Basic,18,False,True
2,A-0a282f,Company_2,DevTools,US,2024-08-27,organic,Basic,1,False,False
3,A-1f0ac7,Company_3,HealthTech,UK,2023-08-27,other,Basic,24,True,False
4,A-ce550d,Company_4,HealthTech,US,2024-10-27,event,Enterprise,35,False,True


In [2]:
START_DATE = pd.Timestamp("2023-01-02")  # Monday
END_DATE = pd.Timestamp("2024-12-31")

LOOKAHEAD_DAYS = 30

# Last snapshot must allow lookahead window inside dataset range
last_snapshot = END_DATE - pd.Timedelta(days=LOOKAHEAD_DAYS)

snapshot_dates = pd.date_range(start=START_DATE, end=last_snapshot, freq="W-MON")

print("Number of snapshot dates:", len(snapshot_dates))
print("First snapshot:", snapshot_dates.min())
print("Last snapshot:", snapshot_dates.max())

Number of snapshot dates: 100
First snapshot: 2023-01-02 00:00:00
Last snapshot: 2024-11-25 00:00:00


In [3]:
churn_min = (
    churn.groupby("account_id", as_index=False)["churn_date"]
    .min()
    .rename(columns={"churn_date": "first_churn_date"})
)

churn_min.head()

,account_id,first_churn_date
0,A-00bed1,2024-01-03
1,A-016043,2024-08-11
2,A-029f69,2024-06-17
3,A-02cd81,2024-04-30
4,A-02fac6,2024-08-21


In [4]:
snapshots = pd.DataFrame({"snapshot_date": snapshot_dates})
accounts_base = accounts[["account_id", "signup_date"]].copy()

# Cross join
accounts_base["key"] = 1
snapshots["key"] = 1
panel = accounts_base.merge(snapshots, on="key").drop(columns=["key"])

# Attach first churn date
panel = panel.merge(churn_min, on="account_id", how="left")

panel.head(), panel.shape

(  account_id signup_date snapshot_date first_churn_date
 0   A-2e4581  2024-10-16    2023-01-02       2024-11-23
 1   A-2e4581  2024-10-16    2023-01-09       2024-11-23
 2   A-2e4581  2024-10-16    2023-01-16       2024-11-23
 3   A-2e4581  2024-10-16    2023-01-23       2024-11-23
 4   A-2e4581  2024-10-16    2023-01-30       2024-11-23,
 (50000, 4))

In [5]:
# Include only snapshots after signup
panel = panel[panel["signup_date"] <= panel["snapshot_date"]].copy()

# Exclude post-churn snapshots (if churn happened before or on snapshot_date)
panel = panel[
    panel["first_churn_date"].isna() | (panel["first_churn_date"] > panel["snapshot_date"])
].copy()

# Tenure
panel["tenure_days"] = (panel["snapshot_date"] - panel["signup_date"]).dt.days

print("Panel rows after filters:", len(panel))
panel.head()

Panel rows after filters: 13096


,account_id,signup_date,snapshot_date,first_churn_date,tenure_days
94,A-2e4581,2024-10-16,2024-10-21,2024-11-23,5
95,A-2e4581,2024-10-16,2024-10-28,2024-11-23,12
96,A-2e4581,2024-10-16,2024-11-04,2024-11-23,19
97,A-2e4581,2024-10-16,2024-11-11,2024-11-23,26
98,A-2e4581,2024-10-16,2024-11-18,2024-11-23,33


In [6]:
panel["label_window_end"] = panel["snapshot_date"] + pd.Timedelta(days=LOOKAHEAD_DAYS)

panel["churn_next_30d"] = np.where(
    panel["first_churn_date"].notna()
    & (panel["first_churn_date"] > panel["snapshot_date"])
    & (panel["first_churn_date"] <= panel["label_window_end"]),
    1,
    0
)

panel["churn_next_30d"].value_counts(normalize=True)

churn_next_30d
0    0.908522
1    0.091478
Name: proportion, dtype: float64

In [7]:
# Prepare subscriptions for as-of join
subs_sorted = subs.sort_values(["account_id", "start_date"]).copy()

# For each snapshot row, we will merge candidate subscriptions and filter active
panel_subs = panel[["account_id", "snapshot_date"]].merge(subs_sorted, on="account_id", how="left")

active_mask = (
    (panel_subs["start_date"] <= panel_subs["snapshot_date"]) &
    (panel_subs["end_date"].isna() | (panel_subs["end_date"] > panel_subs["snapshot_date"]))
)

panel_subs_active = panel_subs[active_mask].copy()

# Choose most recent active subscription by start_date
panel_subs_active = panel_subs_active.sort_values(
    ["account_id", "snapshot_date", "start_date"], ascending=[True, True, False]
)

current_sub = panel_subs_active.drop_duplicates(subset=["account_id", "snapshot_date"], keep="first")

# Keep only the state fields we want
sub_state_cols = [
    "account_id", "snapshot_date", "plan_tier", "seats", "is_trial",
    "billing_frequency", "auto_renew_flag", "mrr_amount", "arr_amount",
    "upgrade_flag", "downgrade_flag"
]

current_sub = current_sub[sub_state_cols].rename(columns={
    "plan_tier": "current_plan_tier",
    "seats": "current_seats",
    "is_trial": "current_is_trial",
    "mrr_amount": "current_mrr",
    "arr_amount": "current_arr",
    "upgrade_flag": "current_upgrade_flag",
    "downgrade_flag": "current_downgrade_flag",
})

current_sub.head()

,account_id,snapshot_date,current_plan_tier,current_seats,current_is_trial,billing_frequency,auto_renew_flag,current_mrr,current_arr,current_upgrade_flag,current_downgrade_flag
82375,A-00bed1,2023-11-20,Basic,61,False,annual,True,1159,13908,True,False
82385,A-00bed1,2023-11-27,Basic,61,False,annual,True,1159,13908,True,False
82395,A-00bed1,2023-12-04,Basic,61,False,annual,True,1159,13908,True,False
82405,A-00bed1,2023-12-11,Basic,61,False,annual,True,1159,13908,True,False
82415,A-00bed1,2023-12-18,Basic,61,False,annual,True,1159,13908,True,False


In [8]:
# Map usage -> account via subscription
usage_with_acct = usage.merge(
    subs[["subscription_id", "account_id"]],
    on="subscription_id",
    how="left"
)

# Defensive: drop any missing account mapping (should be none)
usage_with_acct = usage_with_acct.dropna(subset=["account_id"]).copy()

def aggregate_usage(window_days: int, col_name: str):
    # Join usage to panel (account_id, snapshot_date)
    tmp = panel[["account_id", "snapshot_date"]].merge(
        usage_with_acct[["account_id", "usage_date", "usage_count"]],
        on="account_id",
        how="left"
    )

    # Keep usage within window: (snapshot_date - window_days, snapshot_date]
    start = tmp["snapshot_date"] - pd.to_timedelta(window_days, unit="D")
    in_window = (tmp["usage_date"] > start) & (tmp["usage_date"] <= tmp["snapshot_date"])

    tmp = tmp[in_window].copy()

    agg = (
        tmp.groupby(["account_id", "snapshot_date"], as_index=False)["usage_count"]
        .sum()
        .rename(columns={"usage_count": col_name})
    )
    return agg

usage_30 = aggregate_usage(30, "usage_events_30d")
usage_90 = aggregate_usage(90, "usage_events_90d")

usage_30.head()

,account_id,snapshot_date,usage_events_30d
0,A-00bed1,2023-11-20,40
1,A-00bed1,2023-11-27,35
2,A-00bed1,2023-12-04,23
3,A-00bed1,2023-12-11,43
4,A-00bed1,2023-12-18,27


In [9]:
tickets_small = tickets[["account_id", "submitted_at", "escalation_flag"]].copy()

def aggregate_tickets(window_days: int, count_col: str, esc_col: str = None):
    tmp = panel[["account_id", "snapshot_date"]].merge(
        tickets_small,
        on="account_id",
        how="left"
    )

    start = tmp["snapshot_date"] - pd.to_timedelta(window_days, unit="D")
    in_window = (tmp["submitted_at"] > start) & (tmp["submitted_at"] <= tmp["snapshot_date"])
    tmp = tmp[in_window].copy()

    out = (
        tmp.groupby(["account_id", "snapshot_date"], as_index=False)
        .agg(
            **{count_col: ("submitted_at", "count")},
            **({esc_col: ("escalation_flag", "sum")} if esc_col else {})
        )
    )
    return out

tickets_30 = aggregate_tickets(30, "tickets_30d")
esc_90 = aggregate_tickets(90, "tickets_90d", esc_col="escalations_90d")[["account_id", "snapshot_date", "escalations_90d"]]

tickets_30.head()

,account_id,snapshot_date,tickets_30d
0,A-00bed1,2023-12-18,1
1,A-00bed1,2023-12-25,1
2,A-00bed1,2024-01-01,1
3,A-00cac8,2023-09-18,1
4,A-00cac8,2023-09-25,1


In [10]:
snapshot_df = panel.merge(current_sub, on=["account_id", "snapshot_date"], how="left")

snapshot_df = snapshot_df.merge(usage_30, on=["account_id", "snapshot_date"], how="left")
snapshot_df = snapshot_df.merge(usage_90, on=["account_id", "snapshot_date"], how="left")

snapshot_df = snapshot_df.merge(tickets_30, on=["account_id", "snapshot_date"], how="left")
snapshot_df = snapshot_df.merge(esc_90, on=["account_id", "snapshot_date"], how="left")

# Fill missing aggregates with 0 (means no activity in window)
for c in ["usage_events_30d", "usage_events_90d", "tickets_30d", "escalations_90d"]:
    snapshot_df[c] = snapshot_df[c].fillna(0).astype(int)

# Drop helper column
snapshot_df = snapshot_df.drop(columns=["label_window_end"])

snapshot_df.head(), snapshot_df.shape

(  account_id signup_date snapshot_date first_churn_date  tenure_days  \
 0   A-2e4581  2024-10-16    2024-10-21       2024-11-23            5   
 1   A-2e4581  2024-10-16    2024-10-28       2024-11-23           12   
 2   A-2e4581  2024-10-16    2024-11-04       2024-11-23           19   
 3   A-2e4581  2024-10-16    2024-11-11       2024-11-23           26   
 4   A-2e4581  2024-10-16    2024-11-18       2024-11-23           33   
 
    churn_next_30d current_plan_tier  current_seats current_is_trial  \
 0               0               Pro            9.0            False   
 1               1               Pro           13.0            False   
 2               1               Pro           66.0             True   
 3               1               Pro           66.0             True   
 4               1               Pro            9.0             True   
 
   billing_frequency auto_renew_flag  current_mrr  current_arr  \
 0           monthly            True        441.0       5292

In [11]:
output_path = "../data/processed/account_snapshots_weekly.csv"
snapshot_df.to_csv(output_path, index=False)

print("Wrote:", output_path)
print("Rows:", len(snapshot_df), "Cols:", snapshot_df.shape[1])

Wrote: ../data/processed/account_snapshots_weekly.csv
Rows: 13096 Cols: 19


In [12]:
print("Churn rate in snapshot table:", snapshot_df["churn_next_30d"].mean())

# Any missing current plan info?
missing_plan = snapshot_df["current_plan_tier"].isna().mean()
print("Share of rows with missing current_plan_tier:", round(missing_plan, 4))

snapshot_df["snapshot_date"].min(), snapshot_df["snapshot_date"].max()

Churn rate in snapshot table: 0.09147831398900427
Share of rows with missing current_plan_tier: 0.1639


(Timestamp('2023-01-02 00:00:00'), Timestamp('2024-11-25 00:00:00'))

In [13]:
snapshot_df["churn_next_30d"].mean()

np.float64(0.09147831398900427)